In [ ]:
#!/usr/bin/env python
import sys
import time
sys.path.append("../pydive")
from pydive.pydive import get_void_catalog_wdensity, interpolate_at_circumcenters
from dive import extend_boundaries_box
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator, interp1d, griddata, NearestNDInterpolator
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numba

import MAS_library as MASL
import smoothing_library as SL
import Pk_library as PKL
import readfof
from Corrfunc.theory.xi import xi

%matplotlib inline

In [ ]:

QUIJOTE_SNAPDIR="/global/cscratch1/sd/zhaoc/Quijote/Halos/fiducial/0/"
QUIJOTE_SNAPNUM=4 # z=0
z_dict = {4:0.0, 3:0.5, 2:1.0, 1:2.0, 0:3.0}
redshift = z_dict[QUIJOTE_SNAPNUM]

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(15, 7))
ax = axes.T.ravel()




#points_raw = pd.read_csv('tests/points.dat', engine='c', delim_whitespace=True,
#                    names=['x', 'y', 'z'], usecols=(0, 1, 2)).values

FoF = readfof.FoF_catalog(QUIJOTE_SNAPDIR, QUIJOTE_SNAPNUM, long_ids=False,
                      swap=False, SFR=False, read_IDs=False)

# get the properties of the halos
points_raw = (FoF.GroupPos/1e3).astype(np.double)            #Halo positions in Mpc/h

points = extend_boundaries_box(points_raw, box_size=1000, cpy_range=80, low_range=0).astype(np.double)
print(pd.DataFrame(points, columns=['x', 'y', 'z']).describe())

ngal = points_raw.shape[0] / 1000**3
print(ngal, flush=True)
print(f"==> Doing tesselation", flush=True)
tess = Delaunay(points)
print("\tDone", flush=True)
print(tess.points[[774795,                                                                                                                                                                                        
                    618318,                                                                                                                                                      
                    431163,                                                                                                                                                                                          
                    771415]])



In [ ]:
simplex_coordinates = points[tess.simplices[:,:], :]
dummy_one = np.array([1], dtype=np.double)
weights = np.lib.stride_tricks.as_strided(dummy_one, (points.shape[0],), (0,))
selection = weights

voids = np.zeros((simplex_coordinates.shape[0], 5), dtype=np.double)
volumes = np.zeros((points.shape[0],), dtype=np.double)
density = np.empty((points.shape[0],), dtype=np.double)
density[:] = np.nan

get_void_catalog_wdensity(simplex_coordinates, 
                            tess.simplices,
                            weights,
                            selection,
                            voids,
                            volumes,
                            density,
                            simplex_coordinates.shape[0], 
                            points.shape[0],
                            ngal,
                            4)
vertices_domain = ((points>0) & (points<1000)).reshape(points.shape[0], 3).all(axis=1)
simplices_domain = ((voids[:,:3]>0) & (voids[:,:3]<1000)).reshape(voids[:,:3].shape[0], 3).all(axis=1)
finite_density_mask = volumes!=0
density -= 1




In [ ]:

interp = LinearNDInterpolator(tess, density)
print(f"==> Interpolating DTFE densities at circumcenters", flush=True)
s = time.time()
density_at_voids_dtfe = interp(voids[:,:3])
print(f"\t Done in {time.time() - s} s", flush=True)
print(np.nanmin(density_at_voids_dtfe), np.nanmax(density_at_voids_dtfe), np.nanmean(density_at_voids_dtfe))
nan_mask = np.isnan(density_at_voids_dtfe)


print(f"==> Interpolating DTFE densities at circumcenters idw", flush=True)
s = time.time()
density_at_voids_dtfe_pydive = np.zeros((tess.simplices.shape[0],), dtype=np.double)
interpolate_at_circumcenters(density,
                             points,
                             tess.simplices,
                             voids[:,:4],
                             density_at_voids_dtfe_pydive,
                             0,
                             16)
print(f"\t Done in {time.time() - s} s", flush=True)
print(np.nanmin(density_at_voids_dtfe_pydive[~nan_mask]), np.nanmax(density_at_voids_dtfe_pydive[~nan_mask]), np.nanmean(density_at_voids_dtfe_pydive[~nan_mask]))
print("\tDone", flush=True)

neighbor_volume = voids[:,4] + voids[tess.neighbors, 4].sum(axis=1)

density_at_voids_neigbor = 4 / neighbor_volume 
density_at_voids_neigbor /= ngal
density_at_voids_neigbor -= 1



In [ ]:
interp_volumes = NearestNDInterpolator(voids[:,:3], density_at_voids_neigbor)
x = np.linspace(0, 1000, 256)
X, Y, Z = np.meshgrid(x, x, x)
density_neighbors_grid  = interp(X.reshape(-1), Y.reshape(-1), Z.reshape(-1))
density_neighbors_grid = np.einsum('ijk->jik', density_neighbors_grid.reshape((256, 256, 256)))

In [ ]:


# density field parameters
grid    = 256    #the 3D field will have grid x grid x grid voxels
BoxSize = 1000.0 #Mpc/h ; size of box
MAS     = 'CIC'  #mass-assigment scheme
verbose = True   #print information on progress

# define 3D density field
density_cic_grid = np.zeros((grid,grid,grid), dtype=np.float32)

# construct 3D density field
MASL.MA(points_raw.astype(np.float32), density_cic_grid, BoxSize, MAS, verbose=verbose)
print(np.mean(density_cic_grid))
# at this point, delta contains the effective number of particles in each voxel
# now compute overdensity and density constrast
density_cic_grid /= np.mean(density_cic_grid, dtype=np.float64);  density_cic_grid -= 1.0



BoxSize = 1000
R       = 15.0  #Mpc.h
grid    = density_cic_grid.shape[0]
Filter  = 'Top-Hat'
threads = 16

# compute FFT of the filter
W_k = SL.FT_filter(BoxSize, R, grid, Filter, threads)

# smooth the field
field_smoothed = SL.field_smoothing(density_cic_grid.astype(np.float32), W_k, threads)
density_cic_grid = field_smoothed



density_dtfe_grid = interp(X.reshape(-1), Y.reshape(-1), Z.reshape(-1))
density_dtfe_grid = np.einsum('ijk->jik', density_dtfe_grid.reshape((256, 256, 256)))


density_at_voids_cic = np.zeros(voids.shape[0], dtype=np.float32)
MASL.CIC_interp(density_cic_grid.astype(np.float32), BoxSize, voids[:,:3].astype(np.float32), density_at_voids_cic)


voids = voids[simplices_domain]
density_at_voids_cic = density_at_voids_cic[simplices_domain]
density_at_voids_dtfe = density_at_voids_dtfe[simplices_domain]
density_at_voids_dtfe_pydive = density_at_voids_dtfe_pydive[simplices_domain]
density_at_voids_neigbor = density_at_voids_neigbor[simplices_domain]
density = density[vertices_domain]



In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(15, 7))
ax = axes.T.ravel()

bins_delta = np.concatenate((-np.logspace(0.1, -5, 101), np.logspace(-5, 3, 201)))
bins_r = np.logspace(0, 2, 101)

cut = 2.4
power_law = cut * (ngal * (bins_delta + 1))**(-0.24)

ax[0].hist2d(density_at_voids_cic, voids[:,3], bins=(bins_delta, bins_r), norm=mcolors.LogNorm())
ax[0].plot(bins_delta, power_law, c='r')
ax[0].set_xlabel("$\delta$ CIC")
ax[0].set_ylabel("$R$")
ax[0].set_xscale('symlog')
ax[0].set_yscale('symlog')
ax[1].hist2d(density_at_voids_dtfe, voids[:,3], bins=(bins_delta, bins_r), norm=mcolors.LogNorm())
ax[1].plot(bins_delta, power_law, c='r')
ax[1].set_xlabel("$\delta$ DTFE")
ax[1].set_ylabel("$R$")
ax[1].set_xscale('symlog')
ax[1].set_yscale('symlog')
ax[2].hist2d(density_at_voids_neigbor, voids[:,3], bins=(bins_delta, bins_r), norm=mcolors.LogNorm())
ax[2].plot(bins_delta, power_law, c='r')
ax[2].set_xlabel("$\delta$ Nb")
ax[2].set_ylabel("$R$")
ax[2].set_xscale('symlog')
ax[2].set_yscale('symlog')

#ax[2].hist2d(density_at_voids_dtfe_pydive, voids[:,3], bins=(bins_delta, bins_r), norm=mcolors.LogNorm())
#ax[2].plot(bins_delta, power_law, c='r')
#ax[2].set_xlabel("$\delta$ DTFE IDW (fast)")
#ax[2].set_ylabel("$R$")
#ax[2].set_xscale('symlog')
#ax[2].set_yscale('symlog')

bins = np.logspace(-3, 4, 200)
density=True
#ax[3].hist((1+density_cic_grid).flatten(), bins=bins, histtype='step', label='cic grid', density=density)
#ax[3].hist((1+density_dtfe_grid).flatten(), bins=bins, histtype='step', label='dtfe grid', density=density)
#ax[3].hist((1+density_neighbors_grid).flatten(), bins=bins, histtype='step', label='Nb grid', density=density)

ax[3].hist((1+density_at_voids_cic), bins=bins, histtype='step', label='cic cc', density=density)
ax[3].hist((1+density_at_voids_dtfe), bins=bins, histtype='step', label='dtfe cc', density=density)
ax[3].hist((1+density_at_voids_neigbor), bins=bins, histtype='step', label='Nb cc', density=density)

ax[3].legend()
ax[3].set_xlabel("$1+\delta$")
ax[3].set_xscale('log')

print((1+density_cic_grid)[:,:,50:70].min(), (1+density_dtfe_grid)[:,:,50:70].min(), (1+density_neighbors_grid)[:,:,50:70].min())
print((1+density_cic_grid)[:,:,50:70].max(), (1+density_dtfe_grid)[:,:,50:70].max(), (1+density_neighbors_grid)[:,:,50:70].max())
vmin = 1e-1
vmax = 1e1
p = ax[4].imshow((1+density_cic_grid)[:,:,50:70].mean(axis=2), norm=mcolors.SymLogNorm(linthresh=1e-8, vmin=vmin, vmax=vmax), label = 'CIC')
ax[4].set_title('CIC')
fig.colorbar(p, ax = ax[4])
ax[4].legend()
p=ax[5].imshow((1+density_dtfe_grid)[:,:,50:70].mean(axis=2), norm=mcolors.SymLogNorm(linthresh=1e-8, vmin=vmin, vmax=vmax), label = 'DTFE')
ax[5].set_title('DTFE')
fig.colorbar(p, ax = ax[5])
ax[5].legend()
p=ax[6].imshow((1+density_neighbors_grid)[:,:,50:70].mean(axis=2), norm=mcolors.SymLogNorm(linthresh=1e-8, vmin=vmin, vmax=vmax), label = 'Nb')
ax[6].set_title('Nb')
fig.colorbar(p, ax = ax[6])
ax[6].legend()


BoxSize = 1000.0 #Mpc/h
MAS     = 'CIC'
axis    = 0
threads = 16

# compute the correlation function
CF_cic     = PKL.Xi(density_cic_grid.astype(np.float32), BoxSize, MAS, axis, threads)
cic_mask = CF_cic.r3D < 200
CF_dtfe     = PKL.Xi(density_dtfe_grid.astype(np.float32), BoxSize, 'None', axis, threads)
dtfe_mask = CF_dtfe.r3D < 200
CF_nb     = PKL.Xi(density_neighbors_grid.astype(np.float32), BoxSize, 'None', axis, threads)
nb_mask = CF_nb.r3D < 200
XCF = PKL.XXi(density_cic_grid.astype(np.float32), density_dtfe_grid.astype(np.float32), BoxSize, ['CIC', 'None'], axis, threads)
cross_mask = XCF.r3D < 200


ax[7].plot(CF_cic.r3D[cic_mask], (CF_cic.r3D**2*CF_cic.xi[:,0])[cic_mask], label='CIC', ls='-')
ax[7].plot(CF_dtfe.r3D[dtfe_mask], (CF_dtfe.r3D**2*CF_dtfe.xi[:,0])[dtfe_mask], label='DTFE', ls='--')
ax[7].plot(CF_nb.r3D[nb_mask], (CF_nb.r3D**2*CF_nb.xi[:,0])[nb_mask], label='Nb', ls='--')
ax[7].plot(XCF.r3D[cross_mask], (XCF.r3D**2*XCF.xi[:,0])[cross_mask], label='Cross', ls=':')
ax[7].legend(loc=0)


Pk_cic = PKL.Pk(density_cic_grid.astype(np.float32), BoxSize, axis, 'CIC', threads)
Pk_dtfe = PKL.Pk(density_dtfe_grid.astype(np.float32), BoxSize, axis, 'None', threads)






fig.tight_layout()

fig.savefig('dtfe.png', dpi=200)

fig.show()


In [ ]:
def compute_2pcf(sample, grid=1024, smooth=True, weights=None, MAS='CIC'):
# number of particles
    Np = sample.shape[0]
    # density field parameters
    grid    = grid    #the 3D field will have grid x grid x grid voxels
    BoxSize = 1000.0 #Mpc/h ; size of box
    MAS     = MAS  #mass-assigment scheme
    verbose = True   #print information on progress

    # particle positions in 3D
        # define 3D density field
    delta = np.zeros((grid,grid,grid), dtype=np.float32)

    # construct 3D density field
    MASL.MA(sample.astype(np.float32), delta, BoxSize, MAS, verbose=verbose, W=weights)

    # at this point, delta contains the effective number of particles in each voxel
    # now compute overdensity and density constrast
    delta /= np.mean(delta, dtype=np.float64);  delta -= 1.0

    
    #delta[np.isnan(delta)]=0

    axis    = 0
    threads = 64

    if smooth:
        R       = 10.0  #Mpc.h
        grid    = delta.shape[0]
        Filter  = 'Top-Hat'
        

        # compute FFT of the filter
        W_k = SL.FT_filter(BoxSize, R, grid, Filter, threads)

        # smooth the field
        field_smoothed = SL.field_smoothing(delta, W_k, threads)

        delta=field_smoothed

    # compute the correlation function
    CF     = PKL.Xi(delta, BoxSize, MAS, axis, threads)
    redge = np.linspace(20, 200, 101)
    rout = 0.5 * (redge[1:] + redge[:-1])
    # get the attributes
    r      = CF.r3D      #radii in Mpc/h
    xi0    = CF.xi[:,0]  #correlation function (monopole)
    xi2    = CF.xi[:,1]  #correlation function (quadrupole)
    xi4    = CF.xi[:,2]  #correlation function (hexadecapole)
    Nmodes = CF.Nmodes3D #number of modes
    mask = (r<200)
    rnorm = np.linspace(50, 70, 10)
    xi_i = interp1d(r, r**2*xi0, kind='cubic')
    o_xi0 = xi_i(rout)/rout**2
    norm = (xi_i(rnorm)/rnorm**2).mean()
    xi_i = interp1d(r, r**2*xi2, kind='cubic')
    o_xi2 = xi_i(rout)/rout**2
    xi_i = interp1d(r, r**2*xi4, kind='cubic')
    o_xi4 = xi_i(rout)/rout**2
    
    

    return rout, o_xi0, o_xi2, o_xi4, norm



In [ ]:
plt.figure(figsize=(10, 10))
#plt.plot(CF_dtfe.r3D[dtfe_mask], (CF_dtfe.r3D**2*CF_dtfe.xi[:,0])[dtfe_mask], label='DTFE', ls='--')

mask = voids[:,3] > 16

#r_bl, xi_bl, _, _, norm_ref = compute_2pcf(voids[mask,:3], grid=256, smooth=True, weights=None, MAS='CIC')
#plt.plot(r_bl, r_bl**2*xi_bl, label=f'R>16')

r_bl, xi_bl, _, _, norm_ref = compute_2pcf(voids[:,:3], grid=256, smooth=True, weights=voids[:,4].astype(np.float32), MAS='CIC')
plt.plot(r_bl, r_bl**2*xi_bl* (norm_ref/norm_ref), label=f'All vol')

r_bl, xi_bl, _, _, norm = compute_2pcf(voids[:,:3], grid=256, smooth=True, weights=neighbor_volume[simplices_domain].astype(np.float32), MAS='CIC')
plt.plot(r_bl, r_bl**2*xi_bl * (norm_ref/norm), label=f'All vol-avg')

#for max_delta in [-0.1, -0.2, -0.3, -0.4, -0.5, -0.7, -0.8]:
for R in [15, 16, 17]:
    #mask = density_at_voids_cic < max_delta
    mask = voids[:,3] > R
    r_, xi_ , _, _, norm = compute_2pcf(voids[mask,:3], grid=256, smooth=True, weights=None, MAS='CIC')
    plt.plot(r_, r_**2*xi_* (norm_ref/norm), label=f'R > {R}')
    
    r_, xi_ , _, _ , norm= compute_2pcf(voids[mask,:3], grid=256, smooth=True, weights=voids[:,4][mask].astype(np.float32), MAS='CIC')
    plt.plot(r_, r_**2*xi_* (norm_ref/norm), label=f'R > {R}, vol', ls = '--')
    
    r_, xi_ , _, _ , norm = compute_2pcf(voids[mask,:3], grid=256, smooth=True, weights=neighbor_volume[simplices_domain][mask].astype(np.float32), MAS='CIC')
    plt.plot(r_, r_**2*xi_* (norm_ref/norm), label=f'R > {R}, avg-vol', ls = ':')
    
plt.legend()
#plt.ylim(-100, 100)

In [ ]:
plt.plot(CF_dtfe.r3D[dtfe_mask], (CF_dtfe.r3D**2*np.gradient(CF_dtfe.xi[:,0]))[dtfe_mask], label='DTFE', ls='--')

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(CF_dtfe.r3D[dtfe_mask], (CF_dtfe.r3D**2*CF_dtfe.xi[:,0])[dtfe_mask], label='DTFE', ls='--')

mask = voids[:,3] > 16

#r_bl, xi_bl, _, _, norm_ref = compute_2pcf(voids[mask,:3], grid=256, smooth=True, weights=None, MAS='CIC')
#plt.plot(r_bl, r_bl**2*xi_bl, label=f'R>16')

r_bl, xi_bl, _, _, norm_ref = compute_2pcf(voids[:,:3], grid=256, smooth=True, weights=voids[:,4].astype(np.float32), MAS='CIC')
plt.plot(r_bl, r_bl**2*xi_bl* (norm_ref/norm_ref), label=f'All vol')

r_bl, xi_bl, _, _, norm = compute_2pcf(voids[:,:3], grid=256, smooth=True, weights=neighbor_volume[simplices_domain].astype(np.float32), MAS='CIC')
plt.plot(r_bl, r_bl**2*xi_bl * (norm_ref/norm), label=f'All vol-avg')

#for max_delta in [-0.1, -0.2, -0.3, -0.4, -0.5, -0.7, -0.8]:
for R in [5, 8, 10]:
    #mask = density_at_voids_cic < max_delta
    mask = voids[:,3] < R
    r_, xi_ , _, _, norm = compute_2pcf(voids[mask,:3], grid=256, smooth=True, weights=None, MAS='CIC')
    plt.plot(r_, r_**2*xi_* (norm_ref/norm), label=f'R < {R}')
    
    r_, xi_ , _, _, norm = compute_2pcf(voids[mask,:3], grid=256, smooth=True, weights=(voids[:,4])[mask].astype(np.float32), MAS='CIC')
    plt.plot(r_, r_**2*xi_* (norm_ref/norm), label=f'R < {R}, vol', ls = '--')
    
    r_, xi_ , _, _, norm = compute_2pcf(voids[mask,:3], grid=256, smooth=True, weights=(neighbor_volume)[simplices_domain][mask].astype(np.float32), MAS='CIC')
    plt.plot(r_, r_**2*xi_* (norm_ref/norm), label=f'R < {R}, avg-vol', ls = ':')
    
plt.legend()
#plt.ylim(-100, 100)

In [ ]:
plt.plot(CF_dtfe.r3D[dtfe_mask], (CF_dtfe.r3D**2*CF_dtfe.xi[:,0])[dtfe_mask], label='DTFE', ls='--')
rbins = np.linspace(0, 200, 41)
r = 0.5 * (rbins[1:] + rbins[:-1])
mask = voids[:,3] > 16
results_xi_baseline = xi(1000, 16, rbins, X=voids[mask,0], Y=voids[mask,1], Z=voids[mask,2], verbose=True)
xi_ = results_xi_baseline['xi']
plt.plot(r, r**2*xi_, label=f'R>16')
for max_delta in [-0.1, -0.2, -0.3, -0.4, -0.5, -0.7, -0.8]:
    mask = density_at_voids_cic < max_delta
    results_xi = xi(1000, 16, rbins, X=voids[mask,0], Y=voids[mask,1], Z=voids[mask,2], verbose=True)
    print(mask.shape)
    xi_ = results_xi['xi']
    plt.plot(r, r**2*xi_, label=f'{max_delta:.1f}')
    
plt.legend()
plt.ylim(-100, 100)

In [ ]:
plt.plot(CF_dtfe.r3D[dtfe_mask], (CF_dtfe.r3D**2*CF_dtfe.xi[:,0])[dtfe_mask], label='DTFE', ls='--')
rbins = np.linspace(0, 200, 41)
r = 0.5 * (rbins[1:] + rbins[:-1])
mask = voids[:,3] > 16
results_xi_baseline = xi(1000, 16, rbins, X=voids[mask,0], Y=voids[mask,1], Z=voids[mask,2], verbose=True)
xi_ = results_xi_baseline['xi']
plt.plot(r, r**2*xi_, label=f'R>16')
for max_delta in [-0.1, -0.2, -0.3, -0.4, -0.5, -0.7, -0.8]:
    mask = density_at_voids_dtfe < max_delta
    results_xi = xi(1000, 16, rbins, X=voids[mask,0], Y=voids[mask,1], Z=voids[mask,2], verbose=True)
    print(mask.shape)
    xi_ = results_xi['xi']
    plt.plot(r, r**2*xi_, label=f'{max_delta:.1f}')
    
plt.legend()
plt.ylim(-100, 100)

In [ ]:

mask = voids[:,3] > 16
results_xi_baseline = xi(1000, 16, rbins, X=voids[mask,0], Y=voids[mask,1], Z=voids[mask,2], verbose=True)
print(mask.shape)
xi_bl = results_xi_baseline['xi']


In [ ]:
mask = voids[:,3] * (ngal * (density_at_voids_dtfe + 1))**(0.238) > 2.37
print(mask.sum())
results_xi = xi(1000, 16, rbins, X=voids[mask,0], Y=voids[mask,1], Z=voids[mask,2], verbose=True)

xi_ = results_xi['xi']


In [ ]:
mask = voids[:,3] * (ngal * (density_at_voids_dtfe_pydive + 1))**(0.238) > 2.37
print(mask.sum())
results_xi_cic = xi(1000, 16, rbins, X=voids[mask,0], Y=voids[mask,1], Z=voids[mask,2], verbose=True)

xi_cic = results_xi_cic['xi']


In [ ]:
plt.plot(CF_dtfe.r3D[dtfe_mask], (CF_dtfe.r3D**2*CF_dtfe.xi[:,0])[dtfe_mask], label='DTFE', ls='--')
plt.plot(r, r**2*xi_bl, label=f'Constant')
plt.plot(r, r**2*xi_, label=f'Local DTFE')
plt.plot(r, r**2*xi_cic, label=f'Local CIC')
plt.legend()

In [ ]:
local_ngal = ngal * (density_at_voids_dtfe + 1)
plt.hist(local_ngal, bins = np.logspace(-5, -1, 101), histtype='step', label='scipy interp')
local_ngal = ngal * (density_at_voids_dtfe_pydive + 1)
plt.hist(local_ngal, bins = np.logspace(-5, -1, 101), histtype='step', label='idw')
local_ngal = ngal * (density_at_voids_cic + 1)
plt.hist(local_ngal, bins = np.logspace(-6, -1, 101), histtype='step', label='cic')
local_ngal = ngal * (density_at_voids_neigbor + 1)
plt.hist(local_ngal, bins = np.logspace(-6, -1, 101), histtype='step', label='nb')
plt.gca().axvline(ngal)
plt.xscale('log')
plt.legend()

In [ ]:
bins = np.linspace(0, 40, 101)
plt.hist(voids[:,3], bins = bins, histtype='step', label='real')
R_mod = 2.47 * (ngal * (density_at_voids_dtfe + 1))**(-0.238)
print(R_mod.mean())
plt.hist(R_mod, bins = bins, histtype='step', label='mod scipy interp')
R_mod = 2.47 * (ngal * (density_at_voids_cic + 1))**(-0.238)
plt.hist(R_mod, bins = bins, histtype='step', label='mod cic')
plt.gca().axvline(16)
plt.legend()